# install prerequisites

In [ ]:
!pip install --user pillow scipy numba

## load an image

In [ ]:
import numpy
from scipy.misc import imread 

mat = imread('sample.jpg').astype('f8')
m, n = mat.shape

In [ ]:
%matplotlib inline
from matplotlib import pyplot
pyplot.imshow(mat, cmap='gray')

# Pure Python Implementation

In [ ]:
def generate_zeros_matrix(m, n):
    return [[0 for _ in range(n)] for _ in range(m)]

In [ ]:
def laplacian_filter_pure_python(mat, retval):
    for c in range(1, n - 1):
        for r in range(1, m - 1):
            retval[r][c] = -4.0*mat[r][c] + mat[r-1][c] + mat[r+1][c] + mat[r][c - 1] + mat[r][c + 1]

In [ ]:
def laplacian_filter_pure_python_row_sweep(mat, retval):
    for r in range(1, m - 1):
        for c in range(1, n - 1):
            retval[r][c] = -4.0*mat[r][c] + mat[r-1][c] + mat[r+1][c] + mat[r][c - 1] + mat[r][c + 1]    

In [ ]:
# convert the numpy image to a list of lists
mat_list_of_lists = [[pixel for pixel in row] for row in mat] 

In [ ]:
# create the matrix that will hold the "filtered" image (with the edges)
mat_edges = generate_zeros_matrix(m, n)

In [ ]:
laplacian_filter_pure_python(mat_list_of_lists, mat_edges)
pyplot.imshow(mat_edges, cmap='gray')

In [ ]:
%timeit laplacian_filter_pure_python(mat_list_of_lists, mat_edges)

In [ ]:
%timeit laplacian_filter_pure_python_row_sweep(mat_list_of_lists, mat_edges)

# Numpy

## iterate over the numpy array with python loops

In [ ]:
def laplacian_filter_numpy_python_loops(mat, retval):
    m, n = mat.shape
    for r in range(1, m - 1):
        for c in range(1, n - 1):
            retval[r][c] = -4.0*mat[r][c] + mat[r-1][c] + mat[r+1][c] + mat[r][c - 1] + mat[r][c + 1]
    return retval

In [ ]:
mat_edges = numpy.zeros_like(mat)
laplacian_filter_numpy_python_loops(mat, mat_edges)
pyplot.imshow(mat_edges, cmap='gray')

In [ ]:
%timeit laplacian_filter_numpy_python_loops(mat, mat_edges)

## numpy operations only (no explicit python loops)

In [ ]:
from functools import reduce
def laplacian_filter_numpy_roll(mat):
    mat_ext = numpy.pad(mat, 1, 'constant')

    rolled = [
        numpy.roll(mat_ext, 1, 0),
        numpy.roll(mat_ext, -1, 0),
        numpy.roll(mat_ext, 1, 1),
        numpy.roll(mat_ext, -1, 1)
    ]
    retval = reduce(numpy.add, rolled)[1:-1, 1:-1] - 4.0*mat

In [ ]:
laplacian_filter_numpy_roll(mat)
pyplot.imshow(mat_edges, cmap='gray')

In [ ]:
%timeit laplacian_filter_numpy_roll(mat)

## numpy operations only (no explicit python loops - lower memory footprint)

In [ ]:
def laplacian_filter_numpy_map_reduce(mat):

    mat_ext = numpy.pad(mat, 1, 'constant')
    
    retval = reduce(
        numpy.add,
        map(lambda args: numpy.roll(mat_ext, shift=args[0], axis=args[1]), [[-1, 0], [1, 0], [-1, 1], [1, 1]])
    )

    retval = retval[1:-1, 1:-1] - 4.0*mat

    return retval

In [ ]:
laplacian_filter_numpy_map_reduce(mat)
pyplot.imshow(mat_edges, cmap='gray')

In [ ]:
%timeit laplacian_filter_numpy_map_reduce(mat)

## pure python code + numba

In [ ]:
from numba import jit, double
import numba

In [ ]:
@jit
def laplacian_filter_numba(mat, retval):
    m, n = mat.shape
    for c in range(1, n - 1):
        for r in range(1, m - 1):
            retval[r, c] = -4.0*mat[r, c] + mat[r-1, c] + mat[r+1, c] + mat[r, c - 1] + mat[r, c + 1]

In [ ]:
@jit
def laplacian_filter_numba_row(mat, retval):
    m, n = mat.shape
    for r in range(1, m - 1):
        for c in range(1, n - 1):
            retval[r, c] = -4.0*mat[r, c] + mat[r-1, c] + mat[r+1, c] + mat[r, c - 1] + mat[r, c + 1]

In [ ]:
laplacian_filter_numba(mat, mat_edges)
pyplot.imshow(mat_edges, cmap='gray')

In [ ]:
%timeit laplacian_filter_numba(mat, mat_edges)

In [ ]:
%timeit laplacian_filter_numba_row(mat, mat_edges)

# Cython

In [ ]:
%%file laplacian_filter_cython.pyx

import numpy as np
cimport numpy as np

def laplacian_filter_numpy_cython(np.ndarray mat, np.ndarray retval):
    cdef int m = mat.shape[0]
    cdef int n = mat.shape[1]
    cdef int r, c

    for r in range(1, m - 1):
        for c in range(1, n - 1):
            retval[r, c] = -4.0*mat[r, c] + mat[r-1, c] + mat[r+1, c] + mat[r, c - 1] + mat[r, c + 1]
    return retval

In [ ]:
%%file setup.py

import numpy
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("laplacian_filter_cython.pyx"),
    include_dirs=[numpy.get_include()]
)


In [ ]:
!python3 setup.py build_ext --inplace

In [ ]:
from laplacian_filter_cython import laplacian_filter_numpy_cython

In [ ]:
laplacian_filter_numpy_cython(mat, mat_edges)
pyplot.imshow(mat_edges, cmap='gray')

In [ ]:
%timeit laplacian_filter_numpy_cython(mat, mat_edges)

# Ctypes

In [ ]:
%%file laplacian_filter.c

#include <stdio.h>
#include <stdlib.h>
//#include <omp.h>

void laplacian_filter(double *mat, const int m, const int n, double *retval)
{
//#pragma omp parallel for  shared(mat, retval)
    for( int r = 1; r < m - 1; r++)
    {
        for( int c = 1; c < n - 1; c++)
        {
            const int i = r*n + c;
            const int i_top = (r+1)*n + c;
            const int i_bottom = (r-1)*n + c;
            const int i_left = r*n + (c-1);
            const int i_right = r*n + (c+1);
           
            retval[i] = -4.0*mat[i] + mat[i_bottom] + mat[i_top] + mat[i_left] + mat[i_right];
        }
    }
}

In [ ]:
!gcc laplacian_filter.c -shared -fPIC -O3 -o laplacian_filter_gcc.so

In [ ]:
!ls -l laplacian_filter_gcc.so

In [ ]:
import ctypes

# load the function from the shared library
clib = ctypes.cdll.LoadLibrary('laplacian_filter_gcc.so')
clib.laplacian_filter.argtypes = [
    ctypes.POINTER(ctypes.c_double),
    ctypes.c_int,
    ctypes.c_int,
    ctypes.POINTER(ctypes.c_double),
]


def laplacian_filter_ctypes_gcc(mat, m, n, retval):
    mat_ptr = mat.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
    retval_ptr = retval.ctypes.data_as(ctypes.POINTER(ctypes.c_double))

    clib.laplacian_filter(mat_ptr, m, n, retval_ptr)

In [ ]:
m, n = mat.shape
mat_flat = mat.flatten()
mat_edges_flat = numpy.zeros_like(mat_flat)

laplacian_filter_ctypes_gcc(mat_flat, m, n, mat_edges_flat)
pyplot.imshow(mat_edges_flat.reshape(m, n), cmap='gray')

In [ ]:
%timeit laplacian_filter_ctypes_gcc(mat_flat, m, n, mat_edges_flat)

In [ ]:
!icc laplacian_filter.c -shared -fPIC -xHOST -limf -o laplacian_filter_icc.so

In [ ]:
import ctypes

# load the function from the shared library
clib = ctypes.cdll.LoadLibrary('laplacian_filter_icc.so')
clib.laplacian_filter.argtypes = [
    ctypes.POINTER(ctypes.c_double),
    ctypes.c_int,
    ctypes.c_int,
    ctypes.POINTER(ctypes.c_double),
]


def laplacian_filter_ctypes_icc(mat, m, n, retval):
    mat_ptr = mat.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
    retval_ptr = retval.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
    
    clib.laplacian_filter(mat_ptr, m, n, retval_ptr)

m, n = mat.shape
mat_flat = mat.flatten()
mat_edges_flat = numpy.zeros_like(mat_flat)

laplacian_filter_ctypes_icc(mat_flat, m, n, mat_edges_flat)
pyplot.imshow(mat_edges_flat.reshape(m, n), cmap='gray')

In [ ]:
%timeit laplacian_filter_ctypes_icc(mat_flat, m, n, mat_edges_flat)

In [ ]:
mat_large = imread('sample_large.jpg').astype('f8')
pyplot.imshow(mat_large, cmap='gray')

In [ ]:
%timeit laplacian_filter_ctypes_gcc(mat_large_flat, m_large, n_large, mat_large_edges_flat)
%timeit laplacian_filter_ctypes_icc(mat_large_flat, m_large, n_large, mat_large_edges_flat)

In [ ]:
mat_edges